In [10]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "CDPassword123"
hostname = "nv-desktop-services.apporto.com"
port = 34742
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, hostname, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '80px'}),
        html.P("Developer: Christopher Disla", style ={'fontWeight': 'bold', 'fontSize': 16, 'marginTop': 5})
    ], style={'textAlign': 'center'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        html.Label("Filter by Rescue Type:"),
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
        ],
        value='All',
        labelStyle={'display': 'block'}
        )
        
    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single',
                         selected_rows=[],
                         style_cell={'textAlign': 'left'},
                         style_table={'overflowX': 'auto'}
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type is None or filter_type == 'All':
        filtered = db.read({})
    else:
        if filter_type == 'Water Rescue':
            breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
            sex = "Intact Female"
            age_min, age_max = 26, 156
        elif filter_type == 'Mountain or Wilderness Rescue':
            breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
            sex = "Intact Male"
            age_min, age_max = 26, 156
        elif filter_type == 'Disaster or Individual Tracking':
            breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
            sex = "Intact Male"
            age_min, age_max = 20, 300
        else:
            #default fallback to all
            return db.read({})
        
        query = {
            "breed": {"$in": breeds},
            "sex_upon_outcome": sex,
            "age_upon_outcome_in_weeks": {"$gte": age_min, "$lte": age_max}
        }
        filtered = db.read(query)
        
    df_filtered = pd.DataFrame.from_records(filtered)
    df_filtered = df_filtered.drop_duplicates(subset=['rec_num'])
    
    if not df_filtered.empty and '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) ==0:
        return html.Div("No data available to display.")
    dff = pd.DataFrame.from_dict(viewData)
    
    # Ensure 'breed' column exists in your dataset
    if 'breed' not in dff.columns:
        return html.Div("Breed data not available.")
    
    fig = px.pie(dff, names='breed', title='Preferred Animals by Breed')
    return dcc.Graph(figure=fig)
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    if viewData is None or selected_rows is None or len(selected_rows) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0]

    lat = dff.iloc[row, 13]
    lon = dff.iloc[row, 14]

    if pd.isnull(lat) or pd.isnull(lon):
        # Return default map centered on Austin TX
        return dl.Map(style={'width': '1000px', 'height': '500px'},
                      center=[30.75, -97.48], zoom=10,
                      children=[dl.TileLayer(id="base-layer-id")])

    # Return map centered on selected animal's location with marker
    return dl.Map(style={'width': '1000px', 'height': '500px'},
                  center=[lat, lon], zoom=13,
                  children=[
                      dl.TileLayer(id="base-layer-id"),
                      dl.Marker(position=[lat, lon],
                                children=[
                                    dl.Tooltip(dff.iloc[row, 4]),  # Breed
                                    dl.Popup([
                                        html.H1("Animal Name"),
                                        html.P(dff.iloc[row, 9])  # Name
                                    ])
                                ])
                  ])


app.run_server(debug=True, port=37778)


Dash app running on http://127.0.0.1:37778/
